In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/courses-meta-data-files/meta_data_combined_cources.csv
/kaggle/input/courses-meta-data-files/course_fall_23.csv
/kaggle/input/courses-meta-data-files/meta_data_course_fall_23.csv
/kaggle/input/courses-meta-data-files/meta_data_course_summer_One_All_24.csv
/kaggle/input/courses-meta-data-files/meta_data_course_summer_Two_24.csv
/kaggle/input/courses-meta-data-files/meta_data_course_spring_24.csv
/kaggle/input/courses-meta-data-files/course_summer_One_All_24.csv
/kaggle/input/courses-meta-data-files/course_spring_24.csv
/kaggle/input/courses-meta-data-files/QnA_data_courses.csv
/kaggle/input/courses-meta-data-files/course_summer_Two_24.csv
/kaggle/input/courses-meta-data-files/Final_data_courses.csv


In [ ]:
import urllib.request
from io import BytesIO
import re
import pypdf
import csv
import warnings

# Filter out FutureWarnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
# pdf_urls = "https://enr-apps.as.cmu.edu/assets/SOC/sched_layout_spring.pdf,https://enr-apps.as.cmu.edu/assets/SOC/sched_layout_summer_1.pdf,https://enr-apps.as.cmu.edu/assets/SOC/sched_layout_summer_2.pdf,https://enr-apps.as.cmu.edu/assets/SOC/sched_layout_fall.pdf"

In [ ]:

# def get_pdf_contents(pdf_url):
#     """Retrieve PDF contents from a given URL."""
#     response = urllib.request.urlopen(pdf_url)
#     if response.status == 200:
#         return response.read()
#     raise Exception("Could not retrieve PDF from the URL.")

# def extract_text(pdf_bytes):
#     """Extract text from given bytes object representing PDF."""
#     pdf = pypdf.PdfReader(BytesIO(pdf_bytes))
#     raw_data = ""
    
#     for page in pdf.pages:
#         raw_data += page.extract_text()

#     return raw_data

# def parse_lines(raw_data):
#     """Parse lines within raw_data and return lists."""
#     line_break_pattern = r"(\n)"
#     extracted_lines = re.split(line_break_pattern, raw_data)

#     parsed_lines = []
#     for line in extracted_lines:
#         paragraph_pattern = r"\A[\u00A0]*$"
#         non_breaking_space_match = re.search(paragraph_pattern, line)
        
#         if non_breaking_space_match is None or not non_breaking_space_match.string[-len(non_breaking_space_match.group(0)):] == '\n':
#             parsed_lines.append(line)
            
#     return parsed_lines

# def save_to_csv(parsed_lines, output_file):
#     """Save parsed lines to specified CSV file."""
#     with open(output_file, 'w', encoding='utf-8', newline='') as f:
#         writer = csv.writer(f)
#         writer.writerows([["Information"]]+[[i] for i in parsed_lines])

# #pdf_urls = "https://www.example.com/pdf1.pdf,https://www.example.com/pdf2.pdf"
# results = []
# for idx, pdf_url in enumerate(pdf_urls.split(",")):
#     print(f"Processing {pdf_url}... ({idx + 1}/{len(pdf_urls.split(','))})")
#     try:
#         pdf_bytes = get_pdf_contents(pdf_url)
#         raw_data = extract_text(pdf_bytes)
#         parsed_lines = parse_lines(raw_data)

#         output_filename = os.path.splitext(os.path.basename(pdf_url))[0] + "_lines.csv"
#         full_output_path = output_filename

#         save_to_csv(parsed_lines, full_output_path)

#         print(f"Successfully processed and saved {full_output_path}\n")
#         results.append((pdf_url, full_output_path))
#     except Exception as e:
#         print(f"Error processing {pdf_url}: {str(e)}\n")
# print("\nFinished processing {} URLs.".format(len(results)))

# HTML

In [ ]:
def remove_integer_strings(value):
    try:
        int_value = int(value)
        return np.nan  # Replace integer strings with NaN
    except ValueError:
        return value
    
def remove_strings(value):
    try:
        # Attempt to convert the string to an integer
        int_value = int(value)
        return value  # Replace integer strings with NaN
    except ValueError:
        return np.nan  

In [ ]:
def preprocessing_courses(df):
    column_to_drop = 10
    df = df.drop(column_to_drop, axis=1)
    df = df.reset_index(drop=True)
    df = df.dropna(how='all', subset=df.columns)
    df.columns = df.iloc[0]
    df = df[1:]
    df = df.reset_index(drop=True)
    df['Department Name'] = df['Course']
    df = df.rename(columns={'Course': 'Course ID'})
    df = df.rename(columns={'Title': 'Subject Title'})
    df['Department Name'] = df['Department Name'].apply(remove_integer_strings)
    df['Course ID'] = df['Course ID'].apply(remove_strings)
    df["Department Name"] = df["Department Name"].fillna(method='ffill')
    column_to_keep = 'Department Name'
    df = df.dropna(how='all', subset=df.columns.difference([column_to_keep]))
    df["Course ID"] = df["Course ID"].fillna(method='ffill')
    df["Subject Title"] = df["Subject Title"].fillna(method='ffill')
    df["Units"] = df["Units"].fillna(method='ffill')
    df = df.reset_index(drop=True)
    df = df.fillna("NA")
    
    return df

In [ ]:
def convert_to_working_days(schedule):
    day_mapping = {
        'S': 'Saturday',
        'U': 'Sunday',
        'R': 'Thursday',
        'F': 'Friday',
        'M': 'Monday',
        'T': 'Tuesday',
        'W': 'Wednesday',
        'NA': 'Day not scheduled',
        'TBA': 'Schedule will be added'
    }

    # Split the input schedule string into individual abbreviations
    if schedule == "TBA":
        return day_mapping[schedule]
    if schedule == "NA":
        return day_mapping[schedule]
    
    abbreviations = [ele for ele in schedule]
    #rint(abbreviations)

    # Use a list comprehension to map abbreviations to working days
    working_days = [day_mapping[abbr] for abbr in abbreviations]

    # Join the working days into a single string
    result = ' '.join(working_days)

    return result

def data_transform(df):
    df["Days"]= df["Days"].apply(convert_to_working_days)
    df[['Bldg', 'Room']] = df['Bldg/Room'].str.split(n=1, expand=True)
    df = df.drop(columns=['Bldg/Room'])
    df['Section'] = df['Lec/Sec'].replace('Lec', 'NA')
    df = df.drop(columns=["Lec/Sec"],axis=1)
    
    return df

# File Path

In [ ]:
html_url=["https://enr-apps.as.cmu.edu/assets/SOC/sched_layout_spring.htm","https://enr-apps.as.cmu.edu/assets/SOC/sched_layout_summer_1.htm","https://enr-apps.as.cmu.edu/assets/SOC/sched_layout_summer_2.htm","https://enr-apps.as.cmu.edu/assets/SOC/sched_layout_fall.htm"]


In [ ]:
def getting_csv_file(url,filename):
    html_filepath = url
    tables = pd.read_html(html_filepath, flavor="lxml")
    data = pd.concat(tables, ignore_index=True)
    df=preprocessing_courses(data)
    df = data_transform(df)
    df.to_csv(filename, index=False)
    return df

In [ ]:
url=html_url[0]
df_course_spring_24= getting_csv_file(url,"course_spring_24.csv")

In [ ]:
url=html_url[1]
df_course_summer_One_All_24= getting_csv_file(url,"course_summer_One_All_24.csv")

In [ ]:
url=html_url[2]
df_course_summer_Two_24= getting_csv_file(url,"course_summer_Two_24.csv")

In [ ]:
url=html_url[3]
df_course_fall_23= getting_csv_file(url,"course_fall_23.csv")

# Generating Meta-data

In [ ]:
def generate_sentence(row):
    template = "In Semester Fall 2023, from the department of {DepartmentName}, the subject titled '{SubjectTitle}' with Course ID {CourseID} and Section {Section} offers {Units} units. The Class meets {Days} between {Begin} and {End} ET. Students attend lectures at the {Location} location,led by experienced instructor {Instructor} located in Building {Bldg}, Room {Room}."

    # Format the received row as a dictionary to easily access values by keys
    formatted_row = dict(zip(df.columns, row))
    #print(formatted_row["Course ID"])

    # Fill the template with the row data
    filled_template = template.format(
        CourseID=formatted_row["Course ID"],
        SubjectTitle=formatted_row["Subject Title"],
        Units=formatted_row["Units"],
        Days=formatted_row["Days"],
        Begin=formatted_row["Begin"],
        End=formatted_row["End"],
        Location=formatted_row["Location"],
        Instructor=formatted_row["Instructor(s)"],
        DepartmentName=formatted_row["Department Name"],
        Bldg=formatted_row["Bldg"],
        Room=formatted_row["Room"],
        Section=(formatted_row["Section"] if pd.notna(formatted_row["Section"]) else ""),
    )

    return filled_template

To edit 

In [ ]:
df =df_course_fall_23
filename="course_fall_23"

In [ ]:
df['Meta-Data'] = df.apply(generate_sentence, axis=1)
df_meta_data = pd.DataFrame(df['Meta-Data'])
#df.to_csv('Course_1_meta_data.csv', index=False)
df_meta_data.to_csv("meta_data_"+filename+".csv", index=False)
df.drop('Meta-Data', axis=1, inplace=True)

# Start here

Reading CSV files with Meta data

In [ ]:
file_path = '/kaggle/input/courses-meta-data-files/course_spring_24.csv'
df_spring = pd.read_csv(file_path)

file_path = '/kaggle/input/courses-meta-data-files/course_summer_One_All_24.csv'
df_summer_1 = pd.read_csv(file_path)

file_path = '/kaggle/input/courses-meta-data-files/course_summer_Two_24.csv'
df_summer_2 = pd.read_csv(file_path)

file_path = '/kaggle/input/courses-meta-data-files/course_fall_23.csv'
df_fall = pd.read_csv(file_path)


In [ ]:
# List of DataFrames
dfs = [df_spring, df_summer_1, df_summer_2, df_fall]

# Concatenate DataFrames vertically (appending rows)
appended_df = pd.concat(dfs, ignore_index=True)

In [ ]:
# MDC =meta_data_course
file_path = '/kaggle/input/courses-meta-data-files/meta_data_course_spring_24.csv'
df_MDC_spring = pd.read_csv(file_path)

file_path = '/kaggle/input/courses-meta-data-files/meta_data_course_summer_One_All_24.csv'
df_MDC_summer_1 = pd.read_csv(file_path)

file_path = '/kaggle/input/courses-meta-data-files/meta_data_course_summer_Two_24.csv'
df_MDC_summer_2 = pd.read_csv(file_path)

file_path = '/kaggle/input/courses-meta-data-files/meta_data_course_fall_23.csv'
df_MDC_fall = pd.read_csv(file_path)

combined_df = pd.concat([df_MDC_spring, df_MDC_summer_1, df_MDC_summer_2, df_MDC_fall], ignore_index=True)
combined_df.to_csv("meta_data_combined_cources.csv", index=False)

In [ ]:
print(df_MDC_spring.shape)
print(df_MDC_summer_1.shape)
print(df_MDC_summer_2.shape)
print(df_MDC_fall.shape)

In [ ]:
9222+475+101+9139

In [ ]:
Courses_df = appended_df
Courses_df["Meta-Data"]=combined_df["Meta-Data"]
Courses_df['Semester_Info'] = Courses_df['Meta-Data'].apply(lambda x: x.split('In Semester ')[1].split(',')[0] if 'In Semester' in x else '')

In [ ]:
Courses_df

In [ ]:
Courses_df.columns

In [ ]:
combined_df.values

# Generating Question and Answer from Meta-Data

In [ ]:
combined_df=pd.read_csv("/kaggle/input/courses-meta-data-files/meta_data_combined_cources.csv")

In [ ]:
"""

Q1: How many units does the {subject title} course offer?
A1: The course titled {subject title} with Course ID {Course ID} has {Units} units.

Q2: Which department offers the {subject title} subject?
A2: The {Department} department offers the {subject title} subject.

Q3: When are the classes scheduled for the {subject title}?
A3: The classes meet on {weekday} between {Begin} and {end} ET for the {subject title}.

Q4: Who is the instructor for the {subject title}?
A4: The {subject title} course is led by instructor {instructor}.

Q5: Where are the classes for {subject title} located on the campus?
A5: The {subject title} classes are located in Building {Bldg}, Room {Room}.

Q6: Which semester does the {subject title} belong to?
A6: The {subject title} is a part of the {semester} semester.

"""

In [ ]:
qna_list = []
# Iterate over each row in Courses_df
for index, row in Courses_df.iterrows():
    
    # Generate 6 questions and answers for each row
    question = f'How many units does the {row["Subject Title"]} course offer?'
    answer = f'The course titled {row["Subject Title"]} with Course ID {row["Course ID"]} has {row["Units"]} units.'
    qna_list.append({"Question": question, "Ref_Answer": answer})
    
    question = f'Which department offers the {row["Subject Title"]} subject?'
    answer = f'The {row["Department Name"]} department offers the {row["Subject Title"]} subject.'
    qna_list.append({"Question": question, "Ref_Answer": answer})
    
    question = f'When are the classes scheduled for the {row["Subject Title"]}?'
    answer = f'The classes meet on {row["Days"]} between {row["Begin"]} and {row["End"]} ET for the {row["Subject Title"]}.'
    qna_list.append({"Question": question, "Ref_Answer": answer})
    
    question = f'Who is the instructor for the {row["Subject Title"]}?'
    answer = f'The {row["Subject Title"]} course is led by instructor {row["Instructor(s)"]}.'
    qna_list.append({"Question": question, "Ref_Answer": answer})
    
    question = f'Where are the classes for {row["Subject Title"]} located on the campus?'
    answer = f'The {row["Subject Title"]} class is located in Building {row["Bldg"]}, Room {row["Room"]}.'
    qna_list.append({"Question": question, "Ref_Answer": answer})
    
    question = f'Which semester does the {row["Subject Title"]} belong to?'
    answer = f'The {row["Subject Title"]} is a part of the {row["Semester_Info"]} semester.'
    qna_list.append({"Question": question, "Ref_Answer": answer})
    
    
print(len(qna_list))
QnA_df = pd.DataFrame(qna_list)
QnA_df['Category'] = 'Courses'
# Reset index of QnA_df
QnA_df.reset_index(drop=True, inplace=True)


# Data for USE

In [3]:
QnA_df= pd.read_csv("/kaggle/input/courses-meta-data-files/QnA_data_courses.csv")
Courses_df= pd.read_csv("/kaggle/input/courses-meta-data-files/Final_data_courses.csv")

In [4]:
QnA_df

,Question,Ref_Answer,Category
0,How many units does the Architecture Design St...,The course titled Architecture Design Studio: ...,Courses
1,Which department offers the Architecture Desig...,The Architecture department offers the Archite...,Courses
2,When are the classes scheduled for the Archite...,The classes meet on Monday Wednesday Friday be...,Courses
3,Who is the instructor for the Architecture Des...,The Architecture Design Studio: Poiesis Studio...,Courses
4,Where are the classes for Architecture Design ...,The Architecture Design Studio: Poiesis Studio...,Courses
...,...,...,...
113617,Which department offers the They Came as Roman...,The StuCo (Student Led Courses) department off...,Courses
113618,When are the classes scheduled for the They Ca...,The classes meet on Thursday between 07:30PM a...,Courses
113619,Who is the instructor for the They Came as Rom...,The They Came as Romans: Roman Civilization 36...,Courses
113620,Where are the classes for They Came as Romans:...,The They Came as Romans: Roman Civilization 36...,Courses


In [5]:
# Define filenames for questions and answers files
questions_filename = "questions.txt"
answers_filename = "answers.txt"

# Open files in append mode
with open(questions_filename, 'a') as questions_file, open(answers_filename, 'a') as answers_file:
    # Iterate over each row in the DataFrame
    for index, row in QnA_df.iterrows():
        question = row['Question']
        answer = row['Ref_Answer']

        # Write question and answer to their respective files
        questions_file.write(question + "\n")
        answers_file.write(answer + "\n")